## preprocess the linespline data

In [4]:
## import support libraries
import pandas as pd
import re
import os
import unittest

In [22]:
## get the original dimensions of the vessel

file_path = r'C:\Users\907932\Desktop\Software\Diffrac_related\Diffrac database\001_linesplan database\Original linesplan database Journee Versluis\Versluis.004'
with open(file_path) as linesplan_info:
    info = linesplan_info.read().split('\n')[1]
    info_pro = info.split(' ')
    print(info_pro1)
    vessel_name = info_pro[0][:-1]
    try:
        float(info_pro1[2])
        vessel_type = info_pro[1]
        vesselLoA = float(info_pro[2])
        vesselBeam = float(info_pro[4])
        vesselDraft = float(info_pro[6])
        vesselDepth = float(info_pro[7][1:-1])
    except:
        vessel_type = info[1] + info_pro[2]
        vesselLoA = float(info_pro[3])
        vesselBeam = float(info_pro[5])
        vesselDraft = float(info_pro[7])
        vesselDepth = float(info_pro[8][1:-1])
    num_cross_sections_pattern = re.compile(r'\d+')
    num_cross_sections_int = int(num_cross_sections_pattern.search(info_pro[3]).group(0))
    
    print(vessel_name)
    print(vessel_type)
    print(vesselDepth)
    print(num_cross_sections)
        
    linesplan_info.close()

['VERSLUIS.004:', 'Container', 'Ship,', '250.00', 'x', '32.00', 'x', '9.10', '(11.00)', 'meter.', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
VERSLUIS.004
EShip,
11.0
250


In [11]:
## get the coordinates of the scatters 



'VERSLUIS.004'